In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time
import os
import sys
sys.path.append('../../')

from data.processing import get_data

import models

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
#from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

import yaml

In [3]:
from os.path import exists, join, splitext

In [4]:
models = {'MCMC':'uncer.yaml','BO':'default.yaml'}

In [6]:
runs = 1
PD = {}
for n in range(runs):
    PD[f'm{n}']= {}
    for model_name,config_filename in models.items():
        config = read_config(config_filename)
        prediction_dict = {}
        prediction_dict['m1'] = single_fitting_cycle(**copy.deepcopy(config['fitting']))
        prediction_dict['m1']['forecasts'] = {}
        prediction_dict['m1']['forecasts']['best'] = get_forecast(prediction_dict, train_fit='m1', 
                                                                model=config['fitting']['model'], 
                                                                forecast_days=config['forecast']['forecast_days'])

        prediction_dict['m1']['trials_processed'] = forecast_all_trials(prediction_dict, train_fit='m1', 
                                                                        model=config['fitting']['model'], 
                                                                        forecast_days=config['forecast']['forecast_days'])
        uncertainty_args = {'predictions_dict': prediction_dict, 'fitting_config': config['fitting'],
                    'forecast_config': config['forecast'], **config['uncertainty']['uncertainty_params']}
        uncertainty = config['uncertainty']['method'](**uncertainty_args)
        prediction_dict['uncertainty_forecasts'] = uncertainty.get_forecast()
        prediction_dict['ensemble_mean_forecast'] = uncertainty.ensemble_mean_forecast
        prediction_dict['m1']['metric']['p_val'] = uncertainty.p_val
        PD[f'm{n}'][model_name] = prediction_dict                                                               
    

  1%|          | 12/1500 [00:00<00:12, 117.11it/s]Performing m1 fit ..
loading from ../../misc/cache/dataframes_ts_27112020_tracker.pkl
splitting data ..
splitting data ..
train
 +-------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|             | 172                 | 173                 | 174                 | 175                 | 176                 |
|-------------+---------------------+---------------------+---------------------+---------------------+---------------------|
| date        | 2020-10-15 00:00:00 | 2020-10-16 00:00:00 | 2020-10-17 00:00:00 | 2020-10-18 00:00:00 | 2020-10-19 00:00:00 |
| state       | Maharashtra         | Maharashtra         | Maharashtra         | Maharashtra         | Maharashtra         |
| district    | Mumbai              | Mumbai              | Mumbai              | Mumbai              | Mumbai              |
| total       | 236513.43           | 238242.43           | 24033

NameError: name 'config1' is not defined

In [13]:
runs = 1
PD = {}
for n in range(runs):
    PD[f'm{n}']= {}
    percentile_dict1 = {}
    percentile_dict2 = {}
    percentile_dict1['m1'] = single_fitting_cycle(**copy.deepcopy(config1['fitting']))
    percentile_dict2['m1'] = single_fitting_cycle(**copy.deepcopy(config2['fitting']))
    percentile_dict1['m1']['forecasts'] = {}
    percentile_dict1['m1']['forecasts']['best'] = get_forecast(percentile_dict1, train_fit='m1', 
                                                            model=config1['fitting']['model'], 
                                                            forecast_days=config1['forecast']['forecast_days'])

    percentile_dict1['m1']['trials_processed'] = forecast_all_trials(percentile_dict1, train_fit='m1', 
                                                                    model=config1['fitting']['model'], 
                                                                    forecast_days=config1['forecast']['forecast_days'])
    percentile_dict2['m1']['forecasts'] = {}
    percentile_dict2['m1']['forecasts']['best'] = get_forecast(percentile_dict2, train_fit='m1', 
                                                            model=config2['fitting']['model'], 
                                                            forecast_days=config2['forecast']['forecast_days'])

    percentile_dict2['m1']['trials_processed'] = forecast_all_trials(percentile_dict2, train_fit='m1', 
                                                                    model=config2['fitting']['model'], 
                                                                    forecast_days=config2['forecast']['forecast_days'])

    uncertainty_args1 = {'predictions_dict': percentile_dict1, 'fitting_config': config1['fitting'],
                    'forecast_config': config1['forecast'], **config1['uncertainty']['uncertainty_params']}
    uncertainty1 = config1['uncertainty']['method'](**uncertainty_args1)
    uncertainty_args2 = {'predictions_dict': percentile_dict2, 'fitting_config': config2['fitting'],
                    'forecast_config': config2['forecast'], **config2['uncertainty']['uncertainty_params']}
    uncertainty2 = config2['uncertainty']['method'](**uncertainty_args2)
    uncertainty_forecasts1 = uncertainty1.get_forecasts()
    uncertainty_forecasts2 = uncertainty2.get_forecasts()
    percentile_dict1['m1']['uncertainty_forecasts'] = uncertainty1.ensemble_mean_forecast
    percentile_dict2['m1']['uncertainty_forecasts'] = uncertainty2.ensemble_mean_forecast
    percentile_dict1['m1']['uncertainty_forecasts'] = uncertainty_forecasts1
    percentile_dict2['m1']['uncertainty_forecasts'] = uncertainty_forecasts2
    percentile_dict1['m1']['metric']['p_val'] = uncertainty1.p_val
    percentile_dict2['m1']['metric']['p_val'] = uncertainty2.p_val
    PD[f'm{n}']['MCMC'] = percentile_dict1
    PD[f'm{n}']['BO'] = percentile_dict2
    
    

  1%|          | 14/1500 [00:00<00:10, 139.28it/s]Performing m1 fit ..
loading from ../../misc/cache/dataframes_ts_27112020_tracker.pkl
splitting data ..
splitting data ..
train
 +-------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|             | 172                 | 173                 | 174                 | 175                 | 176                 |
|-------------+---------------------+---------------------+---------------------+---------------------+---------------------|
| date        | 2020-10-15 00:00:00 | 2020-10-16 00:00:00 | 2020-10-17 00:00:00 | 2020-10-18 00:00:00 | 2020-10-19 00:00:00 |
| state       | Maharashtra         | Maharashtra         | Maharashtra         | Maharashtra         | Maharashtra         |
| district    | Mumbai              | Mumbai              | Mumbai              | Mumbai              | Mumbai              |
| total       | 236513.43           | 238242.43           | 24033

KeyboardInterrupt: 

## Comparing best loss from MCMC vs BO

In [11]:
run_names = []
model_names = ['MCMC','BO']
for i in range(runs):
    run_names.append(f'm{i}')

In [ ]:
for run_name in run_names:
    

In [ ]:
m = np.array(mcmc_perc_loss)
b = np.array(bo_perc_loss)


In [ ]:
m = np.mean(m,axis = 0)
b = np.mean(b,axis = 0)

In [ ]:
PD['perc_loss'] = {}
PD['perc_loss']['mcmc'] = m
PD['perc_loss']['bo'] = b

In [ ]:
import pickle as pkl
with open('../../misc/predictions/predictions_dict_perc_20.pickle', 'wb') as handle:
    pkl.dump(PD, handle)


In [ ]:
with open('../../misc/predictions/predictions_dict_perc_20.pickle', 'rb') as handle:
    PD = pkl.load(handle)

In [ ]:
plt.figure(figsize= (15,7))
mc, = plt.plot(key,m,markersize=12,label='MCMC',color='b')
bo, = plt.plot(key,b,markersize=12,label = 'BO',color='r')
plt.legend(handles = [mc,bo])
plt.xlabel('Percentiles')
plt.ylabel('Percentile Loss in Total compartment')

In [ ]:
m.mean()

In [ ]:
b.mean()

In [ ]:
import numpy as np
np.linspace(1,99,40)